# Exercise 2: Perceptron Learning and Maximum Margin Classification

## Preamble
The following code downloads and imports all necessary files and modules into the virtual machine of Colab. Please make sure to execute it before solving this exercise. This mandatory preamble will be found on all exercise sheets.

In [ ]:
import sys, os
if 'google.colab' in sys.modules:
  if os.getcwd() == '/content':
    !git clone 'https://github.com/inb-uni-luebeck/cs4405.git'
    os.chdir('cs4405')

import numpy as np
from matplotlib import pyplot as plt
from utils import utils_2 as utils

Cloning into 'cs4405'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 92 (delta 11), reused 28 (delta 8), pack-reused 61
Unpacking objects: 100% (92/92), 202.54 MiB | 12.70 MiB/s, done.


## Exercise 2.1: Perceptron Learning
Given $L$ training `samples` $\boldsymbol{x}_i \in \mathbb{R}^{1 \times N}$ and its class `labels` $s_i \in \left\{ -1 , 1 \right\}$, we want to train a single artificial neuron, i.e. make it to automatically learn its `weights` $\boldsymbol{w} \in \mathbb{R}^{1 \times N}$ and its `threshold` $\theta \in \mathbb{R}$, such that

$$
\begin{equation}
    \sigma \left( \boldsymbol{x}_{i} \boldsymbol{w}^{T} - \theta \right) = s_{i}, \; \forall i=1,\dots,L
\end{equation}
$$

holds. The sigmoid function is defined as

$$
\begin{equation}
    \sigma(x) = 
        \left\{ 
            \begin{array}{rl}
                1, & \text{if } x \geq 0 \\
                -1, & \text{else}
            \end{array} 
        \right.
\end{equation}
$$

The `weights` $\boldsymbol{w}$ represents a normal vector of a linear hyperplane and `threshold` $\theta$ represents its (by $\lVert \boldsymbol{w} \rVert$ scaled) distance to the origin.

To simplify learning, we apply the *threshold trick*, i.e. we extend the `weights` $\boldsymbol{\hat{w}} \in \mathbb{R}^{1 \times (N + 1)}$ by an additional component in the *first* dimension that represents the `threshold` $\theta$. The `samples` are likewise extended by a component with constant value $-1$ in the *first* dimension (see `help(np.column_stack)`). In this way, for an extended `sample` $\boldsymbol{\hat{x}}_{i} \in \mathbb{R}^{1 \times (N + 1)}$ the output of the neuron can be written as

$$
\begin{equation}
  y_{i} = \sigma \left( \boldsymbol{\hat{x}}_{i} \boldsymbol{\hat{w}}^{T} \right)
\end{equation}.
$$

During each learning `epoch` the given $L$ training `samples` are presented to the artificial neuron in *random order*. We use the perceptron learning rule to adapt the extended `weights` $\boldsymbol{\hat{w}}_{t}$ to $\boldsymbol{\hat{w}}_{t+1}$

$$
\begin{equation}
    \boldsymbol{\hat{w}}_{t+1} = \boldsymbol{\hat{w}}_{t} + \varepsilon (s_{i} - y_{i}) \boldsymbol{\hat{x}}_{i}
\end{equation}
$$

where

$$
\begin{equation}
    y_{i} = \sigma \left( \boldsymbol{\hat{x}}_{i} \boldsymbol{\hat{w}}^{T}_{t} \right)
\end{equation}.
$$

Here, $\varepsilon \in \mathbb{R}^{+}$ denotes the `learning_rate` and $\boldsymbol{\hat{x}}_{i}$ a randomly selected extended training sample.

**Tasks**:
* Implement the function `learn_perceptron` in Python and test it on the training set `data_2.npz`. Apply your perceptron implementation several times to the example training set (start with a `learning_rate` $\varepsilon=0.01$).

**Programming Hints**:
- In each `epoch` of the perceptron learning process a randomly selected training `sample` (see `help(np.random.permutation)`) with class `label` is classified. Then, the `weights` are modified according to the learning rule.
- A single learning epoch might not be sufficient for obtaining a correct classification of all training samples. In this case, further learning epochs should be performed until a correct classification is obtained.
- Due to time limitations, the function `animation.animate` has a parameter `max_frames` which animates only the first `max_frames` steps of the learning process.

In [40]:
def learn_perceptron(samples, labels, learning_rate, epochs):
  
  # TODO n_samples: number of training samples / n_features: number of features -> 179 & 2
  n_samples, n_features = samples.shape
  
  # TODO: initialize extended weight vector (threshold included) randomly
  weights = np.random.rand(n_features+1)

  # TODO: extend samples by '-1' column (threshold trick)
  samples = np.c_[np.ones(n_samples), samples]
  # samples = np.column_stack((samples, np.full((n_samples, 1), -1)))

  for epoch in range(epochs):
    
    # TODO: generate randomly permuted index array
    indexes = np.random.permutation(n_samples)
        
    # iterate through all indexes in the index array
    for index in indexes:

      # TODO: select training sample and corresponding class label according to generated random permutation
      sample = samples[index]
      label = labels[index]
        
      # TODO: classify selected training sample with current weights
      # classification = 1 if sample @ weights >= 0 else -1

      activation = np.dot(sample, weights)
      classification = 1 if activation >= 0 else -1
        
      # TODO: adapt weights, i.e. apply perceptron learning rule
      weights += learning_rate * (label - classification) * sample

      # yield threshold, weight vector, and current sample
      yield {'threshold': weights[0],
              'weights': weights[1:],
              'samples': [sample]}

samples, labels = utils.load_data('data/data_2.npz')
animation = utils.Animation(samples, labels)
generator = learn_perceptron(samples, labels, 
                             learning_rate=0.01, 
                             epochs=1)
animation.animate(generator,
                  max_frames=100)

## Exercise 2.2: The DoubleMinOver Learning Rule
From the lecture, you know that the DoubleMinOver (DMO) learning rule can be used for *maximum margin* classification. The DMO algorithm is summarized below. Note in particular that the `weights` $\boldsymbol{w} \in \mathbb{R}^{1 \times N}$ (i.e. the threshold trick is *not* applied) and that an explicit `threshold` $\theta \in \mathbb{R}$ is used, which is computed after learning has been completed.



for $t=1$ to $t_{\max}$
> $\boldsymbol{x}^{+}_{\min} = \underset{\boldsymbol{x}_i \in X^{+}}{\operatorname{argmin}} s_{i} \boldsymbol{x}_{i} \boldsymbol{w}^{T} \left( X^{+} = \left\{ \boldsymbol{x}_{i} \mid s_{i} = 1 \right\} \right)$  
> $\boldsymbol{x}^{-}_{\min} = \underset{\boldsymbol{x}_i \in X^{-}}{\operatorname{argmin}} s_{i} \boldsymbol{x}_{i} \boldsymbol{w}^{T} \left( X^{-} = \left\{ \boldsymbol{x}_{i} \mid s_{i} = -1 \right\} \right)$  
> $\boldsymbol{w} = \boldsymbol{w} + \left( \boldsymbol{x}^{+}_{\min} - \boldsymbol{x}^{-}_{\min} \right)$

$\theta = \frac{\left(\boldsymbol{x}^{+}_{\min} + \boldsymbol{x}^{-}_{\min}\right) \boldsymbol{w}^{T}}{2}$

**Tasks**:
- Implement the DoubleMinOver algorithm in Python.
- Test your implementation on the training data set `data_2.npz`.

**Questions**:
- Compare your DMO learning results with your perceptron learning results. Run both learning algorithms several times. What differences do you observe in the behaviour of the two algorithms?

**Answers**:
- 


In [ ]:
# TODO: implement the double-min-over learning rule
def learn_dmo(samples, labels, epochs):

  # TODO n_features: number of features
  n_features = 

  # TODO: initialize weights (threshold not! included) randomly
  weights = 

  for epoch in range(epochs):
        
    # TODO: extract training samples of class +1
    samples_pos = 
    
    # TODO: get sample_pos_min
    sample_pos_min =

    # TODO: extract training samples of class -1
    samples_neg = 

    # TODO: get sample_neg_min
    sample_neg_min = 

    # TODO: adapt weight vector, i.e. apply DMO learning rule
    weights = 

    # TODO: calculate threshold
    threshold = 
        
    # yield threshold, weight vector, and current samples
    yield {'threshold': threshold,
            'weights': weights,
            'samples': [sample_pos_min, sample_neg_min]}

samples, labels = utils.load_data('data/data_2.npz')
animation = utils.Animation(samples, labels)
generator = learn_dmo(samples, labels, 
                    epochs=50)
animation.animate(generator,
                  max_frames=50)